# Load Data


In [29]:
# import these modules
import nltk
nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

print("rocks :", lemmatizer.lemmatize("rocks"))
print("corpora :", lemmatizer.lemmatize("corpora"))

# a denotes adjective in "pos"
print("better :", lemmatizer.lemmatize("corpora rocks cats", pos="a"))


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\epeki\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\epeki\AppData\Roaming\nltk_data...


rocks : rock
corpora : corpus
better : corpora rocks cats


[nltk_data]   Package wordnet is already up-to-date!


In [30]:
# This dataset contains ~25k tweets in English.
import requests

url = 'https://raw.githubusercontent.com/mhassanist/nlp_text_datasets/ea689be7f27906d0e27b112bc11311d40967b5a3/mini_twitter.txt'
response = requests.get(url)

In [31]:
with open('mini_twitter.txt', 'wb') as file:
    file.write(response.content)

In [32]:
# Open the file and loads its content
text = ''
with open('mini_twitter.txt', 'r', encoding='utf-8') as content_file:
    text = content_file.read()

In [33]:
# Make sure your data is loaded correctly.
# Task 1: Print the first 100 character of the text

# YOUR CODE HERE
print(text[:100])


How are you? Btw thanks for the RT. You gonna be in DC anytime soon? Love to see you. Been way, way 


# Import required libraries

In [34]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.activations import softmax

# Tokenization

In [35]:
# Task 2.1: Convert the data to lowercase and split it into separate sentences. Store the result in the corpus variable
corpus = text.lower().split("\n")

print(corpus[0]) # how are you? btw thanks for the rt. you gonna be in dc anytime soon? love to see you. been way, way too long.


how are you? btw thanks for the rt. you gonna be in dc anytime soon? love to see you. been way, way too long.


In [36]:
# Task 2.2: Convert all sentences from the text corpus into n-gram sequences of tokens (numeric representations of words)

tokenizer = Tokenizer()

# Fit it on the text
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1


In [37]:
# Task 2.3 : # Use the tokenizer to convert each sentence to n-gram sequence
input_sequences = []

# YOUR CODE HERE
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(input_sequences[1]) # [54, 24, 5]


[56, 22, 5]


# LSTM Model

In [38]:
# Task 3: Create LSTM model, compile it and fit it.
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

x, labels = input_sequences[:,:-1], input_sequences[:,-1]

y = labels


In [39]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len - 1))
model.add(Bidirectional(LSTM(100, return_sequences=True)))
model.add(Dropout(0.5))
model.add(LSTM(100))
model.add(Dense(total_words, activation='softmax'))

adam = Adam(learning_rate=0.01)


c:\Users\epeki\scoop\apps\python\current\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [42]:
# compile the model
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(x, y, epochs=50, steps_per_epoch=500, verbose=1)


Epoch 1/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 646s 1s/step - accuracy: 0.0472 - loss: 7.1532
Epoch 2/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 41us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/50


c:\Users\epeki\scoop\apps\python\current\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


500/500 ━━━━━━━━━━━━━━━━━━━━ 638s 1s/step - accuracy: 0.0722 - loss: 6.7301
Epoch 4/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 24us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 636s 1s/step - accuracy: 0.0855 - loss: 6.4999
Epoch 6/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 637s 1s/step - accuracy: 0.0939 - loss: 6.3403
Epoch 8/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 4us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 9/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 644s 1s/step - accuracy: 0.1015 - loss: 6.2029
Epoch 10/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 31us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 11/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 633s 1s/step - accuracy: 0.1072 - loss: 6.0865
Epoch 12/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 16us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 13/50
500/500 ━━━━━━━━━━━━━━━━━━━━ 623s 1s/step - accuracy: 0.1120 - loss: 5.9802
Epoch 14/50


# Model Prediction

In [45]:
# Complete the following prediction function

# Prediction function
def predict_next_word(text):
    predicted_word = ''


    # YOUR CODE HERE

    token_list = tokenizer.texts_to_sequences([text])[0]
    token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
    predicted = model.predict(token_list, verbose=0)
    predicted_word_index = np.argmax(predicted, axis=1)[0]
    predicted_word = tokenizer.index_word[predicted_word_index]


    return predicted_word


# test the model
print(predict_next_word("how are")) # you

print(predict_next_word("what is your")) # favorite

print(predict_next_word("what do you want to")) # become


you
favorite
be


# Play with it !

In [48]:
text = "let's" # starting text
for _ in range(10): # generate the next 10 words
  next = predict_next_word(text)
  text = text + " " + next

print(text)


let's go to the oscars of the oscars of the masses
